In [ ]:
### Get API Key

import requests
from StringIO import StringIO
from lxml import etree

panorama = "panorama.foo.ltd"
url = "https://%s/api/" % panorama
username = "admin"
password = "Password1"

querystring = {"type":"keygen","user":username,"password":password}

headers = {
    'cache-control': "no-cache",
    }

#### Submit http GET request to panorama with the given params, disable cert verification 

response = requests.request("GET", url, headers=headers, params=querystring, verify=False)

#### Normal response is an XML string, code below processes XML to extract key using xpath
#### XML format for response is:
#### <response status = 'success'>
####   <result>
####      <key>QWEERTYUI12345abcdefg=</key>
####    </result>
#### </response>

tree = etree.parse(StringIO(response.text))
key = tree.xpath("/response/result/key")[0].text


In [ ]:
### Get entire panorama XML config file

import requests
from StringIO import StringIO
from lxml import etree

querystring = {
    "type"    : "config",
    "action"  : "show",
    "key"     : key
}

headers = {
    'cache-control': "no-cache",
    }

response = requests.request("GET", url, headers=headers, params=querystring, verify=False)
tree = etree.parse(StringIO(response.text))
root = tree.getroot()

print(response.text)

In [ ]:
#### To get a config object's xpath, specify a unique config object name and assign to addr_name object below
#### Xpath path //entry will search all entry objects in the XML DOM, with the modifier @name specifying unique 
#### for this element. Ideally this xpath should only return one object if the name is indeed unique.
#### The etree getpath function returns the absolute xpath path for the object that can be used for subsequent
#### API calls

addr_name = "Some Unique address object name"
find_xpath = "//entry[@name='%s']" % addr_name
obj = root.xpath(find_xpath)
xpath = tree.getpath(obj[0])
print xpath

#### Sample output: /response/result/config/devices/entry/device-group/entry[5]/address/entry[14]

In [ ]:
#### Function below is to encapsulate the API call to add an address object
#### with the given path and element fields
#### An args_list argument can be passed to iterate over a templated style 
#### element parameter as shown below


def add_address_objects(url, key, path, headers, element, args_list=[]):
    querystring = {
        "key"      : key,
        "type"     : "config",
        "action"   : "set",
        "xpath"    : path,
        "element"  : element
    }
    
    if args_list:
        for args in args_list:
            querystring["element"] = element % args
            response = requests.request("GET", url, headers=headers, params=querystring, verify=False)
    else:
        response = requests.request("GET", url, headers=headers, params=querystring, verify=False)

element_tmp = """
<entry name="%s">
<ip-netmask>192.168.200.%s</ip-netmask>
<tag>
<member>Foo</member>
</tag>
<description>%s</description>
</entry>
"""

#### The args_list is shown below as a manually created list of fixed-length tuples
#### The args list can also be created dynammically, it all depends on your needs and creativity!

args_list = [
    ("Foo_Address_1", "1", "Foo_Address_1"),
    ("Foo_Address_2", "2", "Foo_Address_2"),    
    ("Foo_Address_3", "3", "Foo_Address_3"),
    ("Foo_Address_30", "30", "Foo_Address_30")    
]

#### Finally the function call to loop over the templated element field with the given args list and send 
#### resulting API call to panorama

add_address_objects(url=url, key=key, path=path, headers=headers, element=element_tmp, args_list=args_list)